In [279]:
import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.model_monitor import CronExpressionGenerator

import json
import jsonlines
import utils
from importlib import reload
import datetime
import time

reload(utils)

<module 'utils' from '/home/sagemaker-user/src/monitoring/utils.py'>

In [237]:
sm_session = sagemaker.Session()

endpoint_name = "index-predictor-endpoint"
feature_group_name = "index-predictor-feature-group-v7"
bucket_name = "team1-index-predictor-bucket"
data_version = "2024-06-26-09-33"

data_capture_prefix = "data-capture"
data_capture_s3_url = f"s3://{bucket_name}/{data_capture_prefix}"

In [238]:
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=CSVSerializer(),
    deserializer=CSVDeserializer(),
)

In [239]:
### Downloading data from feature store

In [240]:
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sm_session)

query = feature_group.athena_query()

query.run(
    query_string=f"""SELECT * FROM "{query.table_name}" WHERE version = '{data_version}'""",
    output_location=f"s3://{bucket_name}/model_monitor/data/",
)

query.wait()

df = query.as_dataframe()

train_df = df[df["type"] == "train"].copy()
validation_df = df[df["type"] == "validation"].copy()
test_df = df[df["type"] == "test"].copy()

selected_test_df = test_df.copy().sample(n=100, random_state=1)

INFO:sagemaker:Query cbb68047-3657-4a1a-9480-2d51d81ecc2e is being executed.
INFO:sagemaker:Query cbb68047-3657-4a1a-9480-2d51d81ecc2e successfully executed.


In [241]:
columns_to_drop = ["type", "version", "write_time", "api_invocation_time", "is_deleted", "datetime"]

df.drop(
    columns=columns_to_drop,
    inplace=True,
)
train_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
validation_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
test_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
selected_test_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
selected_test_no_target = selected_test_df.drop(columns=["close_target"])

In [243]:
test_df_no_target = test_df.drop(columns=["close_target"])

In [244]:
predictions = predictor.predict(test_df_no_target)
flat_predictions = [item for sublist in predictions for item in sublist]

In [245]:
test_df['probabilities'] = flat_predictions
test_df['probabilities'] = test_df['probabilities'].astype(float)
test_df['predictions'] = (test_df['probabilities'] > 0.5).astype(int)

In [204]:
test_df.to_csv("tmp/test_df.csv", index=False)

test_df_s3_uri = S3Uploader.upload("tmp/test_df.csv", f's3://{bucket_name}/model_monitor/model_baseline/test_df.csv')

In [206]:
model_monitor = ModelQualityMonitor(
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=sm_session
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [212]:
model_baseline_job_name = f"index-predictor-model-baselining-{strftime('%d-%H-%M-%S', gmtime())}"

model_baseline_job = model_monitor.suggest_baseline(
    baseline_dataset=test_df_s3_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri = f's3://{bucket_name}/model_monitor/model_baseline/results',
    problem_type="BinaryClassification",
    inference_attribute= "predictions", # The column in the dataset that contains predictions
    probability_attribute= "probabilities", # The column in the dataset that contains probabilities
    ground_truth_attribute= "close_target", # The column in the dataset that contains ground truth labels
    job_name=model_baseline_job_name,
)

model_baseline_job.wait(logs=False)

INFO:sagemaker:Creating processing-job with name index-predictor-model-baselining-27-07-32-25


............................................................!

In [213]:
latest_model_baseline_job = model_monitor.latest_baselining_job
pd.DataFrame(latest_model_baseline_job.suggested_constraints().body_dict["binary_classification_constraints"]).T

,threshold,comparison_operator
recall,0.812834,LessThanThreshold
precision,0.520548,LessThanThreshold
accuracy,0.509804,LessThanThreshold
true_positive_rate,0.812834,LessThanThreshold
true_negative_rate,0.176471,LessThanThreshold
false_positive_rate,0.823529,GreaterThanThreshold
false_negative_rate,0.187166,GreaterThanThreshold
auc,0.494652,LessThanThreshold
f0_5,0.560886,LessThanThreshold
f1,0.634656,LessThanThreshold


In [214]:
pd.DataFrame(latest_model_baseline_job.baseline_statistics().body_dict["binary_classification_metrics"]["confusion_matrix"])

,0,1
0,30,35
1,140,152


In [215]:
### Generate endpoint traffic and ingest ground truth data into it

In [268]:
!aws s3 rm {data_capture_s3_url} --recursive

utils.generate_endpoint_traffic(predictor, selected_test_no_target)

delete: s3://team1-index-predictor-bucket/data-capture/index-predictor-endpoint/AllTraffic/2024/06/27/07/57-20-240-1acbb23e-7022-4d8e-bc0c-f7384561fc93.jsonl


100%|██████████| 100/100 [00:01<00:00, 83.71it/s]


In [269]:
variant_name = sm.describe_endpoint(EndpointName=predictor.endpoint_name)["ProductionVariants"][0]["VariantName"]
ground_truth_upload_s3_url = f"s3://{bucket_name}/ground_truth_data/{predictor.endpoint_name}/{variant_name}"
ground_truth_upload_s3_url

's3://team1-index-predictor-bucket/ground_truth_data/index-predictor-endpoint/AllTraffic'

In [270]:
latest_data_capture_s3_url = utils.get_latest_data_capture_s3_url(bucket_name, data_capture_prefix)
latest_data_capture_prefix = '/'.join(latest_data_capture_s3_url.split('/')[3:])

Found 1 files in s3://team1-index-predictor-bucket/data-capture
Latest data capture S3 url: s3://team1-index-predictor-bucket/data-capture/index-predictor-endpoint/AllTraffic/2024/06/27/08


In [274]:
predictions = selected_test_df['close_target'].astype(int)
counter = 0

def generate_ground_truth_with_id(inference_id):
    global counter
    result = predictions.iloc[counter % len(selected_test_df)]
    counter += 1
    
    # format required by the merge container.
    return {
        "groundTruthData": {
            "data": int(result),
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(inference_id), # eventId must correlate with the eventId in the data capture file
        },
        "eventVersion": "0",
    }
    

def upload_ground_truth(ground_truth_upload_s3_url, file_name, records, upload_time):
    target_s3_uri = f"{ground_truth_upload_s3_url}/{upload_time:%Y/%m/%d/%H}/{file_name}"
    number_of_records = len(records.split('\n'))
    print(f"Uploading {number_of_records} records to {target_s3_uri}")
    
    S3Uploader.upload_string_as_file_body(records, target_s3_uri)
    
    return target_s3_uri

In [275]:
capture_files = utils.get_file_list(bucket_name, latest_data_capture_prefix)

assert capture_files, f"No capture data files found in {latest_data_capture_prefix}. Generate endpoint traffic and wait until capture data appears in the bucket!"

ln = 0

# For each capture data file get the eventIds and generate correlated ground truth labels
for f in capture_files:
    f_name = f.split('/')[-1]
    
    print(f"Downloading {f}")
    S3Downloader.download(f"s3://{bucket_name}/{f}", "./tmp")
    
    print(f"Reading inference ids from the file: ./tmp/{f_name}")
    with jsonlines.open(f"./tmp/{f_name}") as reader: 
        ground_truth_records = "\n".join([
            json.dumps(r) for r in [generate_ground_truth_with_id(l["eventMetadata"]["eventId"]) for l in reader]
        ])
        # for l in reader:
        #     ln += 1
    lastest_ground_truth_s3_uri = upload_ground_truth(ground_truth_upload_s3_url, f"gt-{f_name}", ground_truth_records, datetime.datetime.utcnow())

Found 2 files in s3://team1-index-predictor-bucket/data-capture/index-predictor-endpoint/AllTraffic/2024/06/27/08
Reading inference ids from the file: ./tmp/03-13-122-ca466ca2-9885-40cb-9ce1-2a2b1e6adbc9.jsonl
Uploading 106 records to s3://team1-index-predictor-bucket/ground_truth_data/index-predictor-endpoint/AllTraffic/2024/06/27/08/gt-03-13-122-ca466ca2-9885-40cb-9ce1-2a2b1e6adbc9.jsonl
Reading inference ids from the file: ./tmp/04-29-125-2aca43e2-729c-49dc-bef9-d3f6036fa2e1.jsonl
Uploading 1 records to s3://team1-index-predictor-bucket/ground_truth_data/index-predictor-endpoint/AllTraffic/2024/06/27/08/gt-04-29-125-2aca43e2-729c-49dc-bef9-d3f6036fa2e1.jsonl


In [276]:
endpoint_input = EndpointInput(
    endpoint_name=predictor.endpoint_name,
    probability_attribute="0",
    probability_threshold_attribute=0.5,
    destination="/opt/ml/processing/input_data",
)

In [281]:
model_mon_schedule_name = "index-predictor-model-monitor-schedule-" + time.strftime(
    "%Y-%m-%d-%H-%M-%S", time.gmtime()
)

model_monitor.create_monitoring_schedule(
    monitor_schedule_name=model_mon_schedule_name,
    endpoint_input=endpoint_input,
    problem_type="BinaryClassification",
    # record_preprocessor_script=f"{record_preprocessor_s3_url}/record_preprocessor.py",
    # post_analytics_processor_script=s3_code_postprocessor_uri,
    output_s3_uri=f"s3://{bucket_name}/model_monitor/monitoring/results",
    ground_truth_input=ground_truth_upload_s3_url,
    constraints=model_monitor.suggested_constraints() if model_monitor.latest_baselining_job else f"s3://{bucket_name}/model_monitor/model_baseline/results/constraints.json",
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: index-predictor-model-monitor-schedule-2024-06-27-08-15-53


In [282]:
while model_monitor.describe_schedule()['MonitoringScheduleStatus'] != "Scheduled":
    print(f"Waiting until model monitoring status becomes Scheduled")
    time.sleep(3)
    
model_monitor.describe_schedule()

Waiting until model monitoring status becomes Scheduled
Waiting until model monitoring status becomes Scheduled
Waiting until model monitoring status becomes Scheduled
Waiting until model monitoring status becomes Scheduled
Waiting until model monitoring status becomes Scheduled


{'MonitoringScheduleArn': 'arn:aws:sagemaker:eu-central-1:567821811420:monitoring-schedule/index-predictor-model-monitor-schedule-2024-06-27-08-15-53',
 'MonitoringScheduleName': 'index-predictor-model-monitor-schedule-2024-06-27-08-15-53',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2024, 6, 27, 8, 15, 54, 344000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 6, 27, 8, 16, 11, 824000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2024-06-27-08-15-53-703',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'index-predictor-endpoint',
 'ResponseMetadata': {'RequestId': '34c9ea16-acac-44bf-82dc-0ed469737dfc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '34c9ea16-acac-44bf-82dc-0ed469737dfc',
   'content-type': 'application/x-amz-json-1.1',
   'content-len